In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
x_train = np.array([
    [0.312, 0.259, 0.400, 0.216],
    [0.138, 0.311, 0.028, 0.189],
    [0.135, 0.531, 0.431, 0.135],
    [0.314, 0.279, 0.420, 0.256], 
    [0.118, 0.321, 0.528, 0.389],
    [0.185, 0.501, 0.435, 0.175],
    [0.310, 0.255, 0.422, 0.256], 
    [0.238, 0.311, 0.128, 0.289],
    [0.136, 0.532, 0.471, 0.235]
])

y_train = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1])

In [3]:
print(x_train.dtype)
print(y_train.dtype)
print(x_train.shape)
print(y_train.shape)

float64
int32
(9, 4)
(9,)


In [4]:
from torch.utils.data import DataLoader, Dataset, TensorDataset

class custom_data(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]

        x = torch.tensor(x, dtype = torch.float32)
        y = torch.tensor(y, dtype = torch.long)

        return x, y

In [5]:
train_dataset = custom_data(x_train, y_train)
train_loader = DataLoader(train_dataset)

#test_dataset = custom_data(x_test, y_test)
#test_loader = DataLoader(test_dataset)

In [6]:
x_in, y_out = next(iter(train_loader))

print(x_in.shape)
print(y_out.shape)

print(x_in.dtype)
print(y_out.dtype)

torch.Size([1, 4])
torch.Size([1])
torch.float32
torch.int64


In [7]:
class Mymodel(nn.Module):
    def __init__(self):
        super(Mymodel, self).__init__()
        self.f1 = nn.Linear(4, 10)
        self.f2 = nn.Linear(10, 2)
        self.act = nn.Sigmoid()

    def forward(self, x):
        out = self.f1(x)
        out = self.act(out)
        out = self.f2(out)

        return out

In [8]:
model = Mymodel()

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [10]:
num_epochs = 10000

for epoch in range(num_epochs):
    for x_batch, y_batch in train_loader:
        outputs = model(x_batch)

        optimizer.zero_grad()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    if (epoch+1) % 1000 == 0:
        print(f'Epoch {epoch+1}, Loss : {loss.item()}')

Epoch 1000, Loss : 0.7680871486663818
Epoch 2000, Loss : 0.6887817978858948
Epoch 3000, Loss : 0.6076360940933228
Epoch 4000, Loss : 0.5490191578865051
Epoch 5000, Loss : 0.5062570571899414
Epoch 6000, Loss : 0.4641436040401459
Epoch 7000, Loss : 0.4157794415950775
Epoch 8000, Loss : 0.36406970024108887
Epoch 9000, Loss : 0.3148966133594513
Epoch 10000, Loss : 0.27198439836502075


In [11]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for x_batch, y_batch in train_loader:
        outputs = model(x_batch)
    
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

print(f'Accuracy : {100 * correct / total}%')

Accuracy : 77.77777777777777%
